<a href="https://colab.research.google.com/github/ST-Random5/data_analysis_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
%load_ext tensorboard

import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/data/data_analysis_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/data/data_analysis_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1 : y_train= to_categorical(y_train)
if y_test.ndim ==1 : y_test= to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classess = y_train.shape[1]



In [0]:
def train_model(X_train, y_train, model, params_fit={}):
  
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size',128),
      epochs = params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  
  y_pred_prob = model_trained.predict(X_test)
  y_pred=np.argmax(y_pred_prob, axis=1)
  

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classess):
  return Sequential([
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape = input_shape),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.1),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.1), 




                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),
                     
                    Dense(1024, activation='relu'),
                    Dropout(0.3),
                     
                    Dense(num_classess, activation='softmax'),
])

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape = input_shape),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),

                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']), 




                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_cnn_block_one']),
                     
                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_cnn_block_two']),
                     
                    Dense(num_classess, activation='softmax'),
])

In [0]:
def func_obj(params):
 model=get_model(params)
 model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
 model.fit(
      X_train, 
      y_train,
      batch_size=128, #int(params.get('batch_size',128)),
      epochs = 5,
      verbose=0
      )
  
 score = model.evaluate(X_test,y_test, verbose=0)
 accuracy=score[1]
 print(params, 'accuracy={}'.format(accuracy))
 return {'loss':-accuracy, 'status': STATUS_OK, 'model':model}



In [33]:
model=get_cnn_v5(input_shape, num_classess)
model_trained=train_model(X_train, y_train, model)

predict(model_trained, X_test, y_test)

Epoch 1/5
35/35 [==============================] - 2s 69ms/step - loss: 3.8103 - accuracy: 0.0703 - val_loss: 3.3336 - val_accuracy: 0.1415
Epoch 2/5
35/35 [==============================] - 1s 20ms/step - loss: 2.9978 - accuracy: 0.2052 - val_loss: 2.3371 - val_accuracy: 0.3717
Epoch 3/5
35/35 [==============================] - 1s 20ms/step - loss: 2.1382 - accuracy: 0.3889 - val_loss: 1.4430 - val_accuracy: 0.5859
Epoch 4/5
35/35 [==============================] - 1s 20ms/step - loss: 1.6196 - accuracy: 0.5073 - val_loss: 1.1033 - val_accuracy: 0.7036
Epoch 5/5
35/35 [==============================] - 1s 20ms/step - loss: 1.1749 - accuracy: 0.6351 - val_loss: 0.8070 - val_accuracy: 0.7889


0.7888888888888889

In [36]:
space={
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',0.3, 0.5),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one',0.3, 0.7),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',0.3, 0.7)
}

best=fmin(
    func_obj,
    space,
    tpe.suggest,
    50,
    Trials()
)


{'batch_size': 160.0, 'dropout_cnn_block_one': 0.5393825793500211, 'dropout_cnn_block_three': 0.3617147605292576, 'dropout_cnn_block_two': 0.3135114568185464}
accuracy=0.2566893398761749
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.5256473768739368, 'dropout_cnn_block_three': 0.4284531267124002, 'dropout_cnn_block_two': 0.3542325688469514}
accuracy=0.30725622177124023
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3010779707400095, 'dropout_cnn_block_three': 0.4940996459161374, 'dropout_cnn_block_two': 0.5753554113541967}
accuracy=0.2142857164144516
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.392060967937939, 'dropout_cnn_block_three': 0.44181433477602616, 'dropout_cnn_block_two': 0.32663434280980047}
accuracy=0.3473922908306122
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.33007531812860824, 'dropout_cnn_block_three': 0.4101320712906132, 'dropout_cnn_block_two': 0.4101212049406441}
accuracy=0.46462586522102356
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.400219283040